## 네이버 쇼핑 리뷰 감성분석

In [1]:
import numpy as np
import pandas as pd

In [2]:
url = 'https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt'
df = pd.read_table(url, names=['score','review'])
df.head()

,score,review
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [3]:
df.review[2]

'아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요.'

- 데이터 전처리

In [4]:
df.score.value_counts()

5    81177
2    63989
1    36048
4    18786
Name: score, dtype: int64

In [5]:
# 평점이 4,5 점은 긍정(1), 나머지는 부정(0)
df.score = df.score.apply(lambda x: 1 if x >= 4 else 0)
df.head(3)

,score,review
0,1,배공빠르고 굿
1,0,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,1,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...


In [6]:
# 결측치 확인
df.isna().sum().sum()

0

In [7]:
# 중복데이터 확인
df.shape, df.review.nunique()

((200000, 2), 199908)

In [8]:
# 중복 데이터 제거
df.drop_duplicates(subset=['review'], inplace=True)
df.shape

(199908, 2)

In [9]:
# 한글 이외의 데이터는 제거
df.review = df.review.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', regex=True).str.strip()

In [10]:
# 한글 이외의 데이터를 제거함으로써 발생하는 ''를 제거
df.review.replace('', np.nan, inplace=True)
df.review.isna().sum()

0

In [11]:
df.to_csv('data/네이버쇼핑리뷰 전처리완료.tsv', sep='\t', index=False)

- 한글 형태소 분석 및 불용어 처리

In [12]:
with open('data/한글불용어100.txt') as st:
    lines = st.readlines()
stop_words = [line.split('\t')[0] for line in lines]

In [13]:
from konlpy.tag import Okt
okt = Okt()

In [14]:
from tqdm import tqdm
reviews = []
for review in tqdm(df.review):
    morphs = okt.morphs(review, stem=True)
    clean_morph_review = ' '.join([morph for morph in morphs if morph not in stop_words])
    reviews.append(clean_morph_review)

100%|██████████| 199908/199908 [15:40<00:00, 212.58it/s]


In [15]:
df['processed'] = reviews
df.to_csv('data/네이버쇼핑리뷰 형태소처리완료.tsv', index=False)

- 데이터셋 분리

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    reviews, df.score.values, stratify=df.score.values, random_state=2023
)

- 피쳐 변환 + 모델 학습/평가

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [19]:
pipeline = Pipeline([
    ('cvect', CountVectorizer()), ('nb', MultinomialNB())
])
params = {'cvect__ngram_range':[(1,1),(1,2)], 'cvect__max_df':[0.95,0.98]}
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3)
%time grid_pipe.fit(X_train, y_train)

CPU times: total: 59 s
Wall time: 54.5 s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('cvect', CountVectorizer()),
                                       ('nb', MultinomialNB())]),
             param_grid={'cvect__max_df': [0.95, 0.98],
                         'cvect__ngram_range': [(1, 1), (1, 2)]},
             scoring='accuracy')

In [20]:
grid_pipe.best_params_

{'cvect__max_df': 0.95, 'cvect__ngram_range': (1, 2)}

In [21]:
pipeline = Pipeline([
    ('cvect', CountVectorizer(ngram_range=(1,2))), ('nb', MultinomialNB())
])
params = {'cvect__max_df': [0.93, 0.95, 0.97]}
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3)
%time grid_pipe.fit(X_train, y_train)

CPU times: total: 1min 1s
Wall time: 59.6 s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('cvect',
                                        CountVectorizer(ngram_range=(1, 2))),
                                       ('nb', MultinomialNB())]),
             param_grid={'cvect__max_df': [0.93, 0.95, 0.97]},
             scoring='accuracy')

In [22]:
grid_pipe.best_params_

{'cvect__max_df': 0.93}

In [23]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.8928907297356784

- 모델 저장

In [24]:
import joblib
joblib.dump(grid_pipe.best_estimator_, 'model/네이버쇼핑리뷰 pipeline.pkl')

['model/네이버쇼핑리뷰 pipeline.pkl']